In [6]:
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
# from classification import logisticRegression
from collections import defaultdict
# import functions

<br><br>
## Parse Yelp Reviews   ->   `([Reviews], [Labels])`

In [7]:
import yelp_parser

def parse_yelp_reviews() -> ([str], [str]):
    return yelp_parser.get_chi_hotel_review_score_list()


reviews, scores = parse_yelp_reviews()

ModuleNotFoundError: No module named 'yelp_parser'

<br> 
## Convert each review into a `TaggedDocument`

In [ ]:
import gensim
from nltk.corpus import stopwords

def get_corpus(reviews: [str], scores: [int]):
    stoplist = stopwords.words('english')
    review_tokens = []
    for review in reviews:
        review_tokens.append([word for word in review.lower().split() if word not in stoplist])
    for i, text in enumerate(review_tokens):
        yield gensim.models.doc2vec.TaggedDocument(text, [scores[i]])

corpus = list(get_corpus(reviews, scores))[:20000]
train_corpus, test_corpus = train_test_split(corpus, test_size=0.25, random_state=42)
print(len(train_corpus))

# from math import floor, ceil
# train_corpus = all_data[0:floor(3*total_num_obs/4)]
# test_corpus = all_data[floor(3*total_num_obs/4):]

<br> 
## Train Doc2Vec Model

In [ ]:
import numpy as np
from gensim.models import Doc2Vec

model = Doc2Vec(window=100, dm=1, vector_size=50, min_count=2)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

train_targets, train_regressors = zip(*[(doc.words, doc.tags[0]) for doc in train_corpus])
test_targets, test_regressors = zip(*[(doc.words, doc.tags[0]) for doc in test_corpus])

X = []
for i in range(len(train_targets)):
    X.append(model.infer_vector(train_targets[i]))

train_x = np.asarray(X)
print(train_x.shape)

<br> 
## Add Unique Labels

In [ ]:
from sklearn import preprocessing
Y = np.asarray(train_regressors)
labelEncoder = preprocessing.LabelEncoder()
labelEncoder.fit(Y)
train_y = labelEncoder.transform(Y)
np.mean(train_y)

In [3]:
# def get_learning_vec(doc2vec_model, tagged_docs):
#     targets, regressors = zip(*[(doc.tags[0], doc2vec_model.infer_vector(doc.words, steps=20)) for doc in tagged_docs])
#     return targets, regressors
    

from sklearn import linear_model
logreg = linear_model.LogisticRegression()
logreg.fit(train_x, train_y)  


test_list = []
for i in range(len(test_targets)):
    test_list.append(model.infer_vector(test_targets[i]))

print("What is test_list::" , test_list)
test_x = np.asarray(test_list)
test_Y = np.asarray(test_regressors)
test_y = labelEncoder.transform(test_Y)


# preds = logreg.predict(test_x)
# np.mean(test_y)

# acc = sum(preds == test_y) / len(test_y)
# print(acc)

# def logistic_regression(x, y, test_fraction=0.25):
#     print("Logistic Regression")
#     print("-------------------\n")
    
#     print("Tagging Documents")
# #     x_train_tagged = list(get_corpus(x_train))
# #     x_test_tagged  = list(get_corpus(x_test))
    
#     print(y_train)
#     print("Creating Doc2Vec Model")
#     doc2vec_model = create_doc2vec_model()
#     doc2vec_model.build_vocab(x_train_tagged)
#     doc2vec_model.train(x_train_tagged, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
    
#     print("Building Learning Vectors")
#     y_vec_train, x_vec_train = get_learning_vec(doc2vec_model, x_train_tagged)
#     y_vec_test,  x_vec_test  = get_learning_vec(doc2vec_model, x_test_tagged)
    
#     print("Creating Classifier")
#     classifier = linear_model.LogisticRegression(penalty='l2', fit_intercept=True)
#     classifier.fit(x_vec_train, y_vec_train)
    
#     print("Training and Predicting")
#     functions.train_classifier_and_evaluate_accuracy_on_training_data(classifier, x_vec_train, y_vec_train)
#     functions.train_classifier_and_evaluate_accuracy_on_testing_data(classifier, x_vec_test, y_vec_test)

NameError: name 'train_x' is not defined